In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Lê o conjunto de dados de treino e teste
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# Fixa a semente aleatória para reprodutibilidade
np.random.seed(7)
tf.random.set_seed(7)

# Converte os dados de texto em sequências de números inteiros
texts_train = df_train['text'].tolist()
texts_test = df_test['text'].tolist()

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts_train)
sequences_train = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)

# Trunca e preenche as sequências de entrada
max_review_length = 2000
X_train = pad_sequences(sequences_train, maxlen=max_review_length)
X_test = pad_sequences(sequences_test, maxlen=max_review_length)

# Define as saídas de treino
y_train = df_train['misogynous'].tolist()

# Converte y_train para um array numpy
y_train = np.array(y_train)

# Define o tamanho da camada de embedding
embedding_vector_length = 32

# Cria um modelo sequencial
model = Sequential()

# Transforma a sequência de entrada em uma representação de embedding de palavras
model.add(Embedding(5000, embedding_vector_length, input_length=max_review_length))

# Adiciona uma camada LSTM
model.add(LSTM(100))

# Adiciona uma camada Dropout para prevenir overfitting
model.add(Dropout(0.2))

# Uma camada totalmente conectada com um único nó para a previsão
model.add(Dense(1, activation='sigmoid'))

# Usa binary_crossentropy como loss function para a classificação
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Imprime um resumo do modelo
print(model.summary())

# Treina o modelo (3 epochs)
model.fit(X_train, y_train, epochs=3, batch_size=64)

# Faz previsões no conjunto de teste
y_pred = model.predict(X_test)

# Converte as previsões em 1 ou 0
y_pred = [int(round(pred[0])) for pred in y_pred]

# Cria o arquivo de submissão
submission_file = pd.DataFrame({'id': df_test['id'], 'misogynous': y_pred})
submission_file.to_csv('submission.csv', index=False)